In [2]:
from analyse_trips import *
import geopandas as gpd
import shapely as shp
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

In [3]:
gdf = load_data("mapstr_2024_04_24.geojson")
#   https://www.youtube.com/watch?v=xu1wRfUHtKg 
plot_interactive_map(gdf)

In [4]:
gdf.head()

In [5]:
# drop column a
gdf.drop(columns="userComment", inplace=True)
gdf.drop(columns="icon", inplace=True)
gdf.head()

In [72]:
visited_departements = set(gdf["departement"])
visited_departements.remove(np.nan)
len(visited_departements)

In [21]:
gdf = gdf.to_crs("EPSG:4087")
world_countries_gdf = world_countries_gdf.to_crs("EPSG:4087")

set(gdf.distance(world_countries_gdf.unary_union))

In [25]:
world_union = world_countries_gdf.unary_union

nearest_countries = []
for point in gdf["geometry"]:
    nearest_geom = shp.ops.nearest_points(point, world_union)[1]
    nearest_country = world_countries_gdf[
        world_countries_gdf["geometry"] == nearest_geom
    ]#["name"].values[0]
    
    nearest_countries.append(nearest_country)
    
nearest_countries

In [48]:
from shapely import STRtree

tree = STRtree(world_countries_gdf["geometry"])
nearest_countries = []
for point in gdf["geometry"]:
    # point = gdf["geometry"].iloc[249]
    nearest_geom = tree.nearest(point)
    # break
    nearest_country = world_countries_gdf.loc[nearest_geom]

    nearest_countries.append(nearest_country)
# world_countries_gdf.loc[nearest_geom]
country = gpd.GeoDataFrame(nearest_countries, index=gdf.index)
